# Infection Rate Calibration: Analyze experiment output

Based on a fixed network model, we sweep over the infection rate parameter  𝛽  to find the one that minimizes a point process distance between realizations of simulated processes and the real time series.

In this notebook, we analyze the result of an experiment

Load libs

In [1]:
%load_ext autoreload
%autoreload 2

In [54]:
from matplotlib import pyplot as plt
%matplotlib inline

plt.rcParams.update({
    "figure.autolayout": False,
    "figure.figsize": (12, 5),
    "figure.dpi": 72,
    "axes.linewidth": 0.8,
    "xtick.major.width": 0.8,
    "xtick.minor.width": 0.8,
    "ytick.major.width": 0.8,
    "ytick.minor.width": 0.8,
    "text.usetex": True,
    "font.serif": "Linux Libertine O",
    "font.size": 16,
    "axes.titlesize": 16,
    "axes.labelsize": 16,
    "legend.fontsize": 14,
    "legend.frameon": True,
    "xtick.labelsize": 14,
    "ytick.labelsize": 14,
    "lines.linewidth": 2.0,
    "lines.markersize": 4,
    "grid.linewidth": 0.4,
    "pgf.texsystem": "xelatex",
    "pgf.preamble": [
        r'\usepackage{fontspec}',
        r'\usepackage{unicode-math}',
        r'\setmainfont{Linux Libertine O}',
        r'\setmathfont{Linux Libertine O}',
    ]
})

import re
import os
import json
import random
import networkx as nx
from pprint import pprint
from collections import Counter, defaultdict

from lib.graph_generation import make_ebola_network
from lib.dynamics import SimulationSIR, PriorityQueue
from lib.dynamics import sample_seeds
from lib import utils
from lib import metrics
from lib.settings import DATA_DIR, PROJECT_DIR

---

First set the experiment directory

In [79]:
EXP_DIR = os.path.join(PROJECT_DIR, 'output', 'calibration-net-startJan01-1')

Extract the time period of the experiment (assumed constant over all jobs in the experiment folder)

In [80]:
param_filename = sorted([f for f in os.listdir(EXP_DIR) if f.startswith('param')])[0]
with open(os.path.join(EXP_DIR, param_filename), 'r') as f:
    param_dict = json.load(f)
start_day_str = param_dict['simulation']['start_day_str']
end_day_str = param_dict['simulation']['end_day_str']
del param_filename, param_dict

print(f"start day: {start_day_str}")
print(f"end day: {end_day_str}")

start day: 2014-01-01
end day: 2014-05-01


---

## Estimation of basic reproduction number $R_0$ 

In [95]:
output_fname_re = re.compile('output-p(\d+)-net(\d+)-sim(\d+)')

output_file_list = [f for f in os.listdir(EXP_DIR) if f.startswith('output')]

result_list = []

for i, output_file in enumerate(output_file_list):
    print(f"\rProcess output file {i+1}/{len(output_file_list)}", end="")
    
    # Load param file
    param_filename = f"param-{'-'.join(output_file.split('-')[1:-2])}.json"
    
    with open(os.path.join(EXP_DIR, param_filename), 'r') as f:
        param_dict = json.load(f)
    
    p_out_dict = param_dict['network']['p_out']
    
    p_idx, net_idx, sim_idx = list(map(int, output_fname_re.match(output_file).groups()))
    
    # Load output file
    with open(os.path.join(EXP_DIR, output_file), 'r') as f:
        output_data = json.load(f)
    
    seed_node_list = np.array(list(set([event[0] for event,_ in output_data['init_event_list']])))
    infector_count = Counter(output_data['infector'])
    node_to_idx = dict(output_data['node_idx_pairs'])
    
    country_count = defaultdict(list)
    for u in seed_node_list:
        u_idx = node_to_idx[u]
        u_country = output_data['country'][u_idx]
        inf_count = infector_count[u_idx]
        country_count[u_country].append(inf_count)


    for country in ['Guinea', 'Liberia', 'Sierra Leone']:
        result_list.append({
            'country': country,
            'p_out': p_out_dict[country],
            'p_idx': p_idx, 
            'net_idx': net_idx,
            'sim_idx': sim_idx,
            'count': country_count[country],
        })
        
res_df = pd.DataFrame(result_list)
del result_list

Process output file 4107/4107

In [98]:
res_df_gn = res_df.groupby(['country', 'p_idx']).agg({'count': list, 'p_out': 'min'})
res_df_gn['r0'] = res_df_gn['count'].apply(np.hstack).apply(np.mean)
res_df_gn

count   p_out        r0
country      p_idx                                                                     
Guinea       211    [[0, 13, 4, 7, 1], [7, 5, 0, 3, 4], [0, 0, 0, ...  0.0045  4.540261
             212    [[4, 10, 1, 2, 1], [0, 4, 26, 5, 7], [3, 15, 3...  0.0045  4.546462
             215    [[2, 4, 1, 1, 12], [0, 4, 1, 2, 3], [4, 21, 0,...  0.0045  4.439227
Liberia      211    [[3, 2, 8, 1, 2], [1, 5, 1, 1, 5], [5, 7, 0, 1...  0.0055  4.132304
             212    [[1, 0, 1, 4, 6], [0, 2, 4, 12, 9], [0, 2, 3, ...  0.0055  4.080410
             215    [[0, 2, 2, 8, 3], [5, 6, 1, 10, 8], [0, 7, 1, ...  0.0055  3.979558
Sierra Leone 211    [[], [], [], [], [], [], [], [], [], [], [], [...  0.0047       NaN
             212    [[], [], [], [], [], [], [], [], [], [], [], [...  0.0049       NaN
             215    [[], [], [], [], [], [], [], [], [], [], [], [...  0.0055       NaN

In [97]:
plt.figure(figsize=(10, 6))

x = 1 / df.index

for i, country in enumerate(['Guinea', 'Liberia', 'Sierra Leone']):
    y_mean = df[country]['mean']
    y_std = df[country]['std']
    
    plt.plot(x, y_mean, c=f'C{i}', label=country)
    plt.fill_between(x, y_mean - y_std, y_mean + y_std, color=f'C{i}', alpha=0.3)

plt.xlabel('$1/\\beta$')
plt.ylabel('$R_0$')
plt.legend()

plt.title('Infection rate calibration\nusing basic reproduction number');

NameError: name 'df' is not defined

<Figure size 720x432 with 0 Axes>

In [37]:
from collections import defaultdict, Counter

def compute_r0_per_country(inf_time_arr, infector_arr, country_arr):
    """
    Compute the basic reproduction number R0 per country for the given data.

    inf_time_arr : array-like (shape: (N,))
        Infection time of each of the N nodes
    infector_arr : array-like (shape: (N,))
        Index of the
    """
    # Count of secondary infections: {node_idx: num of infected neighbors}
    infector_count = Counter(infector_arr)
    # Indices of infected nodes
    infected_node_indices = np.where(np.array(inf_time_arr) < np.inf)[0]
    # Initialize the list of number of secondary infections per country
    country_count = defaultdict(list)
    # For each infected node, add its number of secondary case to its country
    for u_idx in infected_node_indices:
        u_country = country_arr[u_idx]
        inf_count = infector_count[u_idx]
        country_count[u_country].append(inf_count)
    country_count = dict(country_count)
    # Compute R0 as the mean number of secondary case for each country
    countru_r0_dict = {country: np.mean(count) for country, count in country_count.items()}
    return countru_r0_dict